#### G2M CLASSIFICATION WITH DECISION TREE CLASSIFIER, BAGGING CLASSIFIER AND TPOT CLASSIFIER

* In this repo, binary classification algorithm is developed to help investment firm to decide whether 'Pink Cab(0)' or 'Yellow Cab(1)' taxi companies would be better for them to invest.  Data in this model is already preprocessed so I've directly started with model selection.
* To do that, I've used TPOT Classifier on data to choose the most optimal ML model along with it's params. Afterwards, I've used Bootstrapt Aggragating to reinforce the model and it's parameters.
* In this task, I've used Accuracy Score to evaluate the model.

#### GETTING KNOW TO DATA

XYZ is a private firm in US. Due to remarkable growth in the Cab Industry in last few years and multiple key players in the market, it is planning for an investment in Cab industry and as per their Go-to-Market(G2M) strategy they want to understand the market before taking final decision. They have 2 potential investment options: Pink Cab or Yellow Cab.
* In dataset, Pink Cab is encoded as 0, while Yellow Cab is encoded as 1.<br>

Moreover,


* NEW YORK NY encoded as 0,
* CHICAGO IL encoded as 1,
* LOS ANGELES CA encoded as 2,
* WASHINGTON DC encoded as 3,
* BOSTON MA encoded as 4,
* SAN DIEGO CA encoded as 5,
* SILICON VALLEY encoded as 6,
* SEATTLE WA encoded as 7,
* ATLANTA GA encoded as 8,
* DALLAS TX encoded as 9,
* MIAMI FL encoded as 10,
* AUSTIN TX encoded as 11,
* ORANGE COUNTY encoded as 12,
* DENVER CO encoded as 13,
* NASHVILLE TN encoded as 14,
* SACRAMENTO CA encoded as 15,
* PHOENIX AZ encoded as 16,
* TUCSON AZ encoded as 17,
* PITTSBURGH PA encoded as 18,
* SAN FRANCISCO CA encoded as 19

In Gender Column,<br>

* Male encoded as 1,
* Female encoded as 0

Lastly, in Payment Method Column, <br>

* Card encoded as 1,
* Cash encoded as 0

In [12]:
import pandas as pd
cabdata = pd.read_csv("C:\\Users\\talfi\\.spyder-py3\\selected.csv")

In [14]:
print("Shape of our data is: ", cabdata.shape)
print("Is there any NAN values in cab data? " ,cabdata.isnull().values.any())
cabdata.head(3)

Shape of our data is:  (10000, 13)
Is there any NAN values in cab data?  False


,Unnamed: 0,KM-Travelled,Price-Charged,Cost-of-Trip,Population,Transaction-ID,Date-of-Travel,Company,City,Customer-ID,Gender,Age,Payment-Mode
0,200157,15.820000,495.190000,214.519200,7350.0,10148662.0,42789.0,1,0,535.0,1,33.0,1
1,179292,34.980000,965.060000,474.328800,10459.0,10063035.0,42588.0,1,0,2429.0,1,32.0,1
2,424136,22.567254,423.443311,286.190113,20679.0,10101350.0,43105.0,1,0,44994.0,0,20.0,0


In [15]:
cabdata.dtypes

Unnamed: 0          int64
KM-Travelled      float64
Price-Charged     float64
Cost-of-Trip      float64
Population        float64
Transaction-ID    float64
Date-of-Travel    float64
Company             int64
City                int64
Customer-ID       float64
Gender              int64
Age               float64
Payment-Mode        int64
dtype: object

Great! Because  data in this model is already preprocessed so we are good to start with model selection. 

#### Splitting our data into Training and Test Sets

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train ,y_test = train_test_split(
    cabdata.drop(columns='Company'),
    cabdata['Company'],
    test_size=0.25,
    random_state=42,
    stratify=cabdata['Company']
)

#### TPOT Classifier
Tree-based Pipeline Optimization Tool, or TPOT for short, is a Python library for automated machine learning. TPOT uses a tree-based structure to represent a model pipeline for a predictive modeling problem, including data preparation and modeling algorithms and model hyperparameters.

In [17]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=3,
    scoring='accuracy',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
tpot.fit(X_train, y_train)

# Print best pipeline steps
print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    # Print idx and transform
    print(f'{idx}. {transform}')

19 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]

_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.

Generation 1 - Current Pareto front scores:

-1	0.8073333333333332	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=gini, DecisionTreeClassifier__max_depth=2, DecisionTreeClassifier__min_samples_leaf=12, DecisionTreeClassifier__min_samples_split=7)

-2	0.8081333333333334	DecisionTreeClassifier(SelectPercentile(input_matrix, SelectPercentile__percentile=55), DecisionTreeClassifier__criterion=gini, DecisionTreeClassifier__max_depth=8, DecisionTreeClassifier__min_samples_leaf=17, DecisionTreeClassifier__min_samples_split=7)
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 f

#### Setting D.T.C. & It's hyperparameters
According to TPOT Classifier, pest pipeline steps are SelectPercentile and DecisionTreeClassifier.<br>
As SelectPercentile is topic for other repo, let's continue with DecisionTreeClassifier by given hyperparameters.

In [18]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=8, min_samples_leaf=17, min_samples_split=7,
                       random_state=42)

In [19]:
from sklearn.metrics import accuracy_score
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
dtc.score(X_test,y_test)

0.792

#### Bootstrapt Aggregating (Bagging)
0.79, hmm; not bad. Let's see whether we can improve it or not with Bootstrapt Aggregation(Bagging).<br>

**Bootstrap aggregating**, also called bagging, is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting.

In [22]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
params = {"n_estimators":[100,200,300,400,500],
          "n_jobs" : [-2,-1,0,1,2]}
grid = GridSearchCV(estimator = BaggingClassifier(),
                    param_grid = params,
                    cv = 3,
                    scoring = "accuracy",
                   verbose = 1, n_jobs = -1)
grid.fit(X_train, y_train)
best_hyperparameters = grid.best_params_
print("Best Hyperparameters for BaggingClassifier are: ", best_hyperparameters)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


C:\Users\talfi\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.79373333 0.79746667        nan 0.79453333 0.7948     0.79573333
 0.79786667        nan 0.79533333 0.79533333 0.79693333 0.7972
        nan 0.79693333 0.7968     0.79733333 0.79786667        nan
 0.79613333 0.79866667 0.79786667 0.79666667        nan 0.7964
 0.7964    ]
  warnings.warn(


Best Hyperparameters for BaggingClassifier are:  {'n_estimators': 400, 'n_jobs': 2}


In [31]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
bc = BaggingClassifier(base_estimator = dtc, n_estimators = 400, n_jobs = 2)
bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Classification Accuracy of Bagging Classifier is: {:.3f}".format(accuracy))

Classification Accuracy of Bagging Classifier is: 0.804


From 0.792 to 0.804. It may not seem so much, but even slight differences in M.L. Models have huge impacts on data!